In [1]:
!pip install transformers torch datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
from transformers import BertForSequenceClassification, BertTokenizer
from torch.utils.data import DataLoader
from datasets import load_dataset
import torch
import torch.nn as nn
import torch.optim as optim

In [3]:
# Load datasets
sst_dataset = load_dataset("glue", "sst2")
quora_dataset = load_dataset("quora")
sts_dataset = load_dataset("glue", "stsb")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/35.3k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/3.11M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/72.8k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/148k [00:00<?, ?B/s]

...

validation-00000-of-00001.parquet:   0%|          | 0.00/151k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/114k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5749 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1500 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1379 [00:00<?, ? examples/s]

In [4]:
# Check class distribution
print(quora_dataset["train"].filter(lambda x: x["is_duplicate"]).num_rows)  # Number of duplicates
print(quora_dataset["train"].filter(lambda x: not x["is_duplicate"]).num_rows)  # Number of non-duplicates


Filter:   0%|          | 0/404290 [00:00<?, ? examples/s]

149263


Filter:   0%|          | 0/404290 [00:00<?, ? examples/s]

255027


In [5]:
from datasets import Dataset
from datasets import concatenate_datasets

def downsample_balanced(dataset, num_samples_per_class=None):
    """
    Downsample the dataset to balance classes.

    Args:
        dataset: The dataset split (e.g., train).
        num_samples_per_class: Optional. Number of examples per class.
                               If None, downsample to the smallest class size.

    Returns:
        A balanced and reduced dataset.
    """
    # Separate the dataset by class
    duplicates = dataset.filter(lambda x: x["is_duplicate"] == True)
    non_duplicates = dataset.filter(lambda x: x["is_duplicate"] == False)

    # Determine the number of samples per class
    if num_samples_per_class is None:
        num_samples_per_class = min(len(duplicates), len(non_duplicates))

    # Downsample each class
    duplicates_sampled = duplicates.shuffle(seed=42).select(range(num_samples_per_class))
    non_duplicates_sampled = non_duplicates.shuffle(seed=42).select(range(num_samples_per_class))

    # Combine and shuffle the two subsets
    balanced_dataset = concatenate_datasets([duplicates_sampled, non_duplicates_sampled]).shuffle(seed=42)

    return balanced_dataset

In [6]:
# Downsample to 80,000 examples per class
reduced_quora_dataset = downsample_balanced(quora_dataset["train"], num_samples_per_class=80000)

Filter:   0%|          | 0/404290 [00:00<?, ? examples/s]

Filter:   0%|          | 0/404290 [00:00<?, ? examples/s]

In [7]:
print(reduced_quora_dataset.num_rows)  # Total examples
print(reduced_quora_dataset.filter(lambda x: x["is_duplicate"]).num_rows)  # Duplicates
print(reduced_quora_dataset.filter(lambda x: not x["is_duplicate"]).num_rows)  # Non-duplicates

160000


Filter:   0%|          | 0/160000 [00:00<?, ? examples/s]

80000


Filter:   0%|          | 0/160000 [00:00<?, ? examples/s]

80000


In [8]:
len(reduced_quora_dataset)

160000

In [9]:
len(sst_dataset['train'])

67349

In [10]:
len(sts_dataset['train'])

5749

In [11]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [12]:
# Tokenization function for SST
def tokenize_function_sst(examples, max_length=128):
    return tokenizer(examples['sentence'], padding="max_length", truncation=True, max_length=max_length)

# Tokenize SST
sst_tokenized = sst_dataset.map(tokenize_function_sst, batched=True)

Map:   0%|          | 0/67349 [00:00<?, ? examples/s]

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

Map:   0%|          | 0/1821 [00:00<?, ? examples/s]

In [13]:
print(sst_tokenized["train"][0])

{'sentence': 'hide new secretions from the parental units ', 'label': 0, 'idx': 0, 'input_ids': [101, 5342, 2047, 3595, 8496, 2013, 1996, 18643, 3197, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [14]:
# Tokenization function for Quora
def tokenize_function_quora(examples, max_length=128):
    # Combine question texts from the 'text' field
    sentences = [' '.join(question_pair['text']) for question_pair in examples['questions']]
    return tokenizer(sentences, padding="max_length", truncation=True, max_length=max_length)

quora_tokenized = reduced_quora_dataset.map(tokenize_function_quora, batched=True, batch_size=100)

Map:   0%|          | 0/160000 [00:00<?, ? examples/s]

In [15]:
from datasets import DatasetDict

# Rename the dataset to have a "train" split
quora_tokenized = DatasetDict({"train": quora_tokenized})

# Verify the result
print(quora_tokenized["train"][0])

{'questions': {'id': [374033, 374034], 'text': ["Why isn't Norway part of the EU?", 'Why has Norway not joined the EU?']}, 'is_duplicate': True, 'input_ids': [101, 2339, 3475, 1005, 1056, 5120, 2112, 1997, 1996, 7327, 1029, 2339, 2038, 5120, 2025, 2587, 1996, 7327, 1029, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [16]:
# Tokenization function for STS
def tokenize_function_sts(examples, max_length=128):
    sentences = [s1 + " " + s2 for s1, s2 in zip(examples['sentence1'], examples['sentence2'])]
    return tokenizer(sentences, padding="max_length", truncation=True, max_length=max_length)

# Tokenize STS
sts_tokenized = sts_dataset.map(tokenize_function_sts, batched=True)

Map:   0%|          | 0/5749 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

Map:   0%|          | 0/1379 [00:00<?, ? examples/s]

In [17]:
print(sts_tokenized["train"][0])

{'sentence1': 'A plane is taking off.', 'sentence2': 'An air plane is taking off.', 'label': 5.0, 'idx': 0, 'input_ids': [101, 1037, 4946, 2003, 2635, 2125, 1012, 2019, 2250, 4946, 2003, 2635, 2125, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [18]:
def create_dataloader(tokenized_dataset, label_col, batch_size=8, shuffle=True):
    # Remove columns dynamically based on dataset
    columns_to_remove = [col for col in ["idx", "sentence", "questions", "sentence1", "sentence2"] if col in tokenized_dataset.column_names]
    return DataLoader(
        dataset=tokenized_dataset.remove_columns(columns_to_remove).with_format("torch"),
        batch_size=batch_size,
        shuffle=shuffle,
        collate_fn=lambda x: {
            "input_ids": torch.stack([d["input_ids"] for d in x]),
            "attention_mask": torch.stack([d["attention_mask"] for d in x]),
            "labels": torch.tensor([int(d[label_col]) for d in x]),  # Convert labels to int
        }
    )



sst_train_loader = create_dataloader(sst_tokenized["train"], "label",shuffle=True)
quora_train_loader = create_dataloader(quora_tokenized["train"], "is_duplicate",batch_size=32,shuffle=True)
sts_train_loader = create_dataloader(sts_tokenized["train"], "label",shuffle=True)

In [19]:
from transformers import BertModel

class MultiTaskBert(nn.Module):
    def __init__(self, pretrained_model_name="bert-base-uncased"):
        super(MultiTaskBert, self).__init__()
        self.bert = BertModel.from_pretrained(pretrained_model_name)

        # Task-specific heads
        self.sst_head = nn.Linear(self.bert.config.hidden_size, 2)  # 2 classes for SST
        self.quora_head = nn.Linear(self.bert.config.hidden_size, 2)  # 2 classes for Quora
        self.sts_head = nn.Linear(self.bert.config.hidden_size, 1)  # Scalar for STS regression

    def forward(self, input_ids, attention_mask, task_name, token_type_ids=None):
        outputs = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
        )
        pooled_output = outputs.pooler_output  # [CLS] token representation

        # Select the task-specific head
        if task_name == "SST":
            logits = self.sst_head(pooled_output)
        elif task_name == "Quora":
            logits = self.quora_head(pooled_output)
        elif task_name == "STS":
            logits = self.sts_head(pooled_output)
        else:
            raise ValueError(f"Unknown task: {task_name}")

        return logits

In [ ]:
from scipy.stats import pearsonr

def train(model, dataloaders, epochs=2, lr=3e-5):
    optimizer = optim.AdamW(model.parameters(), lr=lr)
    device = "cuda" if torch.cuda.is_available() else "cpu"

    model.train()
    for epoch in range(epochs):
        print(f"Epoch {epoch+1}/{epochs}")
        for task_name, dataloader in dataloaders.items():
            print(f"Training on {task_name}")
            total_loss = 0
            correct_preds = 0
            total_preds = 0
            all_preds = []
            all_labels = []

            for batch in dataloader:
                optimizer.zero_grad()
                inputs = {key: val.to(device) for key, val in batch.items() if key != "labels"}
                labels = batch["labels"].to(device)

                # Forward pass
                logits = model(**inputs, task_name=task_name)

                # Use appropriate loss function
                if task_name == "STS":
                    loss_fn = nn.MSELoss()
                    logits = logits.squeeze(-1)  # Remove extra dimension
                    loss = loss_fn(logits, labels.float())

                    # Collect predictions and labels for Pearson correlation
                    all_preds.extend(logits.detach().cpu().numpy())
                    all_labels.extend(labels.detach().cpu().numpy())
                else:
                    loss_fn = nn.CrossEntropyLoss()
                    loss = loss_fn(logits, labels)

                    # Accuracy calculation
                    preds = torch.argmax(logits, dim=1)
                    correct_preds += (preds == labels).sum().item()
                    total_preds += labels.size(0)

                # Backpropagation
                loss.backward()
                optimizer.step()

                total_loss += loss.item()

            avg_loss = total_loss / len(dataloader)

            if task_name == "STS":
                # Compute Pearson correlation
                correlation, _ = pearsonr(all_preds, all_labels)
                print(f"Task: {task_name}, Average Loss: {avg_loss:.4f}, Pearson Correlation: {correlation:.4f}")
            else:
                accuracy = correct_preds / total_preds
                print(f"Task: {task_name}, Average Loss: {avg_loss:.4f}, Accuracy: {accuracy:.4f}")

In [20]:
model = MultiTaskBert()
model = model.to("cuda" if torch.cuda.is_available() else "cpu")

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [ ]:
dataloaders = {
    "SST": sst_train_loader
}

train(model, dataloaders)

Epoch 1/3
Training on SST
Task: SST, Average Loss: 0.2276, Accuracy: 0.9130
Epoch 2/3
Training on SST
Task: SST, Average Loss: 0.1362, Accuracy: 0.9533
Epoch 3/3
Training on SST
Task: SST, Average Loss: 0.1073, Accuracy: 0.9639


In [ ]:
dataloaders = {
    "STS": sts_train_loader
}

train(model, dataloaders)

Epoch 1/3
Training on STS
Task: STS, Average Loss: 2.1777, Pearson Correlation: 0.0701
Epoch 2/3
Training on STS
Task: STS, Average Loss: 2.0965, Pearson Correlation: 0.1562
Epoch 3/3
Training on STS
Task: STS, Average Loss: 1.8409, Pearson Correlation: 0.3734


In [ ]:
dataloaders = {
    "Quora": quora_train_loader,
}

train(model, dataloaders)

Epoch 1/2
Training on Quora
Task: Quora, Average Loss: 0.3360, Accuracy: 0.8535
Epoch 2/2
Training on Quora
Task: Quora, Average Loss: 0.2123, Accuracy: 0.9160


## CLN IMPLEMENTATION

In [21]:
import torch
import torch.nn as nn

class ConditionalLayerNorm(nn.Module):
    def __init__(self, hidden_size, num_tasks):
        super(ConditionalLayerNorm, self).__init__()
        # Default LayerNorm parameters
        self.weight = nn.Parameter(torch.ones(hidden_size))
        self.bias = nn.Parameter(torch.zeros(hidden_size))

        # Task-specific parameters
        self.task_weight = nn.Parameter(torch.zeros(num_tasks, hidden_size))
        self.task_bias = nn.Parameter(torch.zeros(num_tasks, hidden_size))

    def forward(self, x, task_id):
        # Calculate mean and variance of input
        mean = x.mean(dim=-1, keepdim=True)
        std = x.std(dim=-1, keepdim=True)

        # Get task-specific weights and biases
        task_weight = self.task_weight[task_id]
        task_bias = self.task_bias[task_id]

        # Adjust weights and biases with task-specific parameters
        effective_weight = self.weight + task_weight
        effective_bias = self.bias + task_bias

        # Apply normalization
        x = (x - mean) / (std + 1e-5)
        return effective_weight * x + effective_bias

In [22]:
class BertLayerWithCLN(nn.Module):
    def __init__(self, bert_layer, hidden_size, num_tasks):
        super(BertLayerWithCLN, self).__init__()
        self.attention = bert_layer.attention
        self.intermediate = bert_layer.intermediate
        self.output = bert_layer.output
        self.cln = ConditionalLayerNorm(hidden_size, num_tasks)

    def forward(self, hidden_states, attention_mask, task_id):
        # Modified to handle potential tuple output from self.attention
        attention_output = self.attention(hidden_states, attention_mask)

        # Check if attention_output is a tuple and unpack if necessary
        if isinstance(attention_output, tuple):
            attention_output = attention_output[0]  # Assuming the first element is the output we need

        # Add and apply Conditional LayerNorm
        attention_output = self.cln(attention_output + hidden_states, task_id)

        intermediate_output = self.intermediate(attention_output)
        layer_output = self.output(intermediate_output, attention_output)
        return layer_output

In [23]:
from transformers import BertModel

class BertWithCLN(nn.Module):
    def __init__(self, pretrained_model_name, num_tasks):
        super(BertWithCLN, self).__init__()
        # Load the pre-trained BERT model
        self.bert = BertModel.from_pretrained(pretrained_model_name)
        hidden_size = self.bert.config.hidden_size

        # Replace embedding layer's LayerNorm with ConditionalLayerNorm
        self.cln_embeddings = ConditionalLayerNorm(hidden_size, num_tasks)

        # Replace encoder layers with BertLayerWithCLN
        self.encoder_layers = nn.ModuleList([
            BertLayerWithCLN(layer, hidden_size, num_tasks) for layer in self.bert.encoder.layer
        ])

        # Task-specific heads
        self.task_heads = nn.ModuleDict({
            "SST": nn.Linear(hidden_size, 2),  # Sentiment classification
            "Quora": nn.Linear(hidden_size, 2),  # Paraphrase detection
            "STS": nn.Linear(hidden_size, 1),  # Semantic similarity regression
        })

    def forward(self, input_ids, attention_mask, task_id, task_name):
        # Embedding layer with ConditionalLayerNorm
        embeddings = self.bert.embeddings(input_ids)
        embeddings = self.cln_embeddings(embeddings, task_id)

        # Encoder layers with CLN
        hidden_states = embeddings
        for layer in self.encoder_layers:
            hidden_states = layer(hidden_states, attention_mask, task_id)

        # Use the correct task-specific head
        task_head = self.task_heads[task_name]
        output = task_head(hidden_states[:, 0, :])  # Use [CLS] token representation
        return output

In [24]:
# Number of tasks (e.g., SST, Quora, STS)
num_tasks = 3

# Initialize minBERT with CLN
model_with_cln = BertWithCLN(pretrained_model_name="bert-base-uncased", num_tasks=num_tasks)

# Move to device
device = "cuda" if torch.cuda.is_available() else "cpu"
model_with_cln.to(device)

BertWithCLN(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_a

In [ ]:
from scipy.stats import pearsonr
def train_sst_sts(model, dataloaders, task_ids, epochs=3, lr=3e-5):
    """
    Train the model on SST and STS tasks.

    Args:
        model: The BertWithCLN model.
        dataloaders: A dictionary of DataLoaders for SST and STS tasks.
        task_ids: A dictionary mapping task names to task IDs.
        epochs: Number of epochs.
        lr: Learning rate.
    """
    optimizer = optim.AdamW(model.parameters(), lr=lr)
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model.train()

    for epoch in range(epochs):
        print(f"Epoch {epoch+1}/{epochs}")
        for task_name, dataloader in dataloaders.items():
            task_id = task_ids[task_name]
            print(f"Training on {task_name}")
            total_loss = 0
            correct_preds, total_preds = 0, 0
            all_preds, all_labels = [], []

            for batch in dataloader:
                optimizer.zero_grad()
                input_ids = batch["input_ids"].to(device)

                # Cast attention mask to float and add an extra dimension
                attention_mask = batch["attention_mask"].to(device).float()
                attention_mask = attention_mask.unsqueeze(1).unsqueeze(2) # Add extra dimensions

                # Expand attention mask to match the sequence length
                attention_mask = attention_mask.expand(-1, attention_mask.size(1), attention_mask.size(2), input_ids.size(1))

                labels = batch["labels"].to(device)

                # Forward pass
                logits = model(input_ids, attention_mask, task_id=task_id, task_name=task_name)

                # Loss and Metrics
                if task_name == "STS":
                    loss_fn = nn.MSELoss()
                    logits = logits.squeeze(-1)
                    loss = loss_fn(logits, labels.float())

                    all_preds.extend(logits.detach().cpu().numpy())
                    all_labels.extend(labels.detach().cpu().numpy())

                elif task_name == "SST":
                    loss_fn = nn.CrossEntropyLoss()
                    loss = loss_fn(logits, labels)
                    preds = torch.argmax(logits, dim=1)
                    correct_preds += (preds == labels).sum().item()
                    total_preds += labels.size(0)

                # Backpropagation
                loss.backward()
                optimizer.step()
                total_loss += loss.item()

            # Metrics
            avg_loss = total_loss / len(dataloader)
            if task_name == "STS":
                correlation, _ = pearsonr(all_preds, all_labels)
                print(f"Task: {task_name}, Average Loss: {avg_loss:.4f}, Pearson Correlation: {correlation:.4f}")
            elif task_name == "SST":
                accuracy = correct_preds / total_preds
                print(f"Task: {task_name}, Average Loss: {avg_loss:.4f}, Accuracy: {accuracy:.4f}")

In [ ]:
# Task IDs
task_ids = {
    "SST": 0,
    "STS": 2,
}

# DataLoaders
dataloaders = {
    "SST": sst_train_loader,
    "STS": sts_train_loader,
}

# Train the model
train_sst_sts(model_with_cln, dataloaders, task_ids, epochs=3, lr=3e-5)

Epoch 1/3
Training on SST
Task: SST, Average Loss: 0.1214, Accuracy: 0.9583
Training on STS
Task: STS, Average Loss: 0.6884, Pearson Correlation: 0.8235
Epoch 2/3
Training on SST
Task: SST, Average Loss: 0.0805, Accuracy: 0.9721
Training on STS
Task: STS, Average Loss: 0.4359, Pearson Correlation: 0.8923
Epoch 3/3
Training on SST
Task: SST, Average Loss: 0.0604, Accuracy: 0.9789
Training on STS
Task: STS, Average Loss: 0.3095, Pearson Correlation: 0.9248


In [27]:
def train_quora(model, dataloader, task_id, epochs=2, lr=3e-5):
    """
    Train the model on the Quora task.

    Args:
        model: The BertWithCLN model.
        dataloader: DataLoader for the Quora task.
        task_id: Task ID for Quora.
        epochs: Number of epochs to train.
        lr: Learning rate.
    """
    optimizer = optim.AdamW(model.parameters(), lr=lr)
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model.train()

    for epoch in range(epochs):
        print(f"Epoch {epoch+1}/{epochs} for Quora")
        total_loss = 0
        correct_preds, total_preds = 0, 0

        for batch in dataloader:
            optimizer.zero_grad()
            input_ids = batch["input_ids"].to(device)

            # Process attention mask
            attention_mask = batch["attention_mask"].to(device).float()
            attention_mask = attention_mask.unsqueeze(1).unsqueeze(2)  # Add extra dimensions
            attention_mask = attention_mask.expand(-1, attention_mask.size(1), attention_mask.size(2), input_ids.size(1))

            labels = batch["labels"].to(device)

            # Forward pass
            logits = model(input_ids, attention_mask, task_id=task_id, task_name="Quora")

            # ***CHANGE***: Select the correct output for binary classification
            logits = logits[:, 1] # Select the probability for class 1


            # Binary Cross-Entropy Loss
            loss_fn = nn.BCEWithLogitsLoss()
            logits = logits.squeeze(-1)  # Remove extra dimension if needed
            loss = loss_fn(logits, labels.float())

            # Accuracy calculation (threshold at 0.5 for binary classification)
            preds = (torch.sigmoid(logits) > 0.5).long()
            correct_preds += (preds == labels).sum().item()
            total_preds += labels.size(0)

            # Backpropagation
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        # Metrics for the epoch
        avg_loss = total_loss / len(dataloader)
        accuracy = correct_preds / total_preds
        print(f"Task: Quora, Average Loss: {avg_loss:.4f}, Accuracy: {accuracy:.4f}")

In [29]:
# Task ID for Quora
quora_task_id = 1

# Train the model
train_quora(model_with_cln, quora_train_loader, task_id=quora_task_id, epochs=2, lr=3e-5)

Epoch 1/2 for Quora
Task: Quora, Average Loss: 0.1873, Accuracy: 0.9285
Epoch 2/2 for Quora
Task: Quora, Average Loss: 0.1176, Accuracy: 0.9573
